In [1]:
import urllib
import urllib.request
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [148]:
binance_base = "https://<>.binance.com"
binance_subdomains = ["api", "api1", "api2", "api3"]

binance_url = binance_base.replace('<>', binance_subdomains[0])

binance_endpoints = {
    'ping': ('GET', '/api/v3/ping'),
    'server_time': ('GET', '/api/v3/time'),
    'exchange_info': ('GET', '/api/v3/exchangeInfo'),
    'order_book': ('GET', '/api/v3/depth', {'symbol': True, 'limit': False}),
    'recent_trades': ('GET', '/api/v3/trades', {'symbol': True, 'limit': False}),
    'average_price': ('GET', '/api/v3/avgPrice', {'symbol': True}),
    'price': ('GET', '/api/v3/ticker/price', {'symbol': False}),
    'best_book_price': ('GET', '/api/v3/ticker/bookTicker', {'symbol': False})
}

In [149]:
def get_price(symbol):
    binance_uri = binance_url + binance_endpoints['average_price'][1] + f'?symbol={symbol}'
    web_url = urllib.request.urlopen(binance_uri)
    data = web_url.read()
    encoding = web_url.info().get_content_charset('utf-8')
    JSON_object = json.loads(data.decode(encoding))
    return {"symbol": symbol, "price": float(JSON_object["price"])}

def get_symbols_prices(symbols):
    prices = []
    for s in symbols:
        prices.append(get_price(s))
    return prices

In [171]:
binance_uri = binance_url + binance_endpoints['exchange_info'][1]

web_url = urllib.request.urlopen(binance_uri)
data = web_url.read()
encoding = web_url.info().get_content_charset('utf-8')
JSON_object = json.loads(data.decode(encoding))

df_symbols = pd.DataFrame(JSON_object['symbols'])
df_symbols = df_symbols.set_index("symbol")
df_symbols.head()

,status,baseAsset,baseAssetPrecision,quoteAsset,quotePrecision,quoteAssetPrecision,baseCommissionPrecision,quoteCommissionPrecision,orderTypes,icebergAllowed,ocoAllowed,quoteOrderQtyMarketAllowed,isSpotTradingAllowed,isMarginTradingAllowed,filters,permissions
symbol,,,,,,,,,,,,,,,,
ETHBTC,TRADING,ETH,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
LTCBTC,TRADING,LTC,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
BNBBTC,TRADING,BNB,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
NEOBTC,TRADING,NEO,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","[SPOT, MARGIN]"
QTUMETH,TRADING,QTUM,8,ETH,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...",True,True,True,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[SPOT]


In [172]:
selected_symbols = ["DOGEEUR", "LTCUSDT", "DOTEUR", "BNBUSDT", "DOTBTC", "ETHUSDT", "LTCBTC", "BTCEUR", 
                    "BNBETH", "DOGEBTC", "LTCBNB", "BNBBTC", "ETHBTC", "EURUSDT", "DOTBNB", "LTCETH", 
                    "LTCEUR", "DOTUSDT", "ETHEUR", "BTCUSDT", "DOGEUSDT", "BNBEUR"]
df_symbols = df_symbols.loc[selected_symbols]
df_symbols = df_symbols.sort_index()

In [173]:
prices = get_symbols_prices(selected_symbols)

In [174]:
df_symbols["price"] = pd.DataFrame().from_dict(prices).set_index("symbol")

In [175]:
#df_symbols["f_PRICE_minPrice"] = df_symbols["filters"].apply(lambda f: [float(x["minPrice"]) for x in f if x["filterType"]=="PRICE_FILTER"][0])
#df_symbols["f_PRICE_tickSize"] = df_symbols["filters"].apply(lambda f: [float(x["tickSize"]) for x in f if x["filterType"]=="PRICE_FILTER"][0])
df_symbols["f_LOT_minQty"] = df_symbols["filters"].apply(lambda f: [float(x["minQty"]) for x in f if x["filterType"]=="LOT_SIZE"][0])
df_symbols["f_LOT_stepSize"] = df_symbols["filters"].apply(lambda f: [float(x["stepSize"]) for x in f if x["filterType"]=="LOT_SIZE"][0])
df_symbols["f_NOTIONA_minNotional"] = df_symbols["filters"].apply(lambda f: [float(x["minNotional"]) for x in f if x["filterType"]=="MIN_NOTIONAL"][0])
df_symbols["f_NOTIONA_applyToMarket"] = df_symbols["filters"].apply(lambda f: [x["applyToMarket"] for x in f if x["filterType"]=="MIN_NOTIONAL"][0])
df_symbols["f_LOT_step_roundto"] = df_symbols["f_LOT_stepSize"].apply(lambda x: int(np.absolute(np.log10(x))))

In [176]:
f_cols = [c for c in df_symbols.columns if c[:2]=="f_"]

In [177]:
base_eur_map = df_symbols[df_symbols["quoteAsset"]=="EUR"][["baseAsset", "price"]].set_index("baseAsset").to_dict()["price"]
df_symbols["eur_price"] = df_symbols["baseAsset"].map(base_eur_map)

In [178]:
def my_ceil(a, ndigits=0):
    return np.true_divide(np.ceil(a * 10**ndigits), 10**ndigits)

In [179]:
df_symbols["min_qty"] = df_symbols["f_NOTIONA_minNotional"] / df_symbols["price"]

df_symbols["min_qty_rounded"] = df_symbols.apply(lambda x: my_ceil(x["min_qty"], ndigits=x["f_LOT_step_roundto"]), axis=1)
df_symbols["min_qty_val_notional"] = df_symbols["min_qty_rounded"] * df_symbols["price"]
df_symbols["min_qty_val_in_eur"] = df_symbols["min_qty_rounded"] * df_symbols["eur_price"]

min_qty_map = df_symbols.groupby("baseAsset")["min_qty_rounded"].max().to_dict()
df_symbols["min_safe_qty"] = df_symbols["baseAsset"].map(min_qty_map)
df_symbols["min_safe_qty_val_in_eur"] =  df_symbols["min_safe_qty"] * df_symbols["eur_price"]

In [180]:
df_symbols[["min_safe_qty", "min_safe_qty_val_in_eur", "f_LOT_minQty", "f_LOT_stepSize", "f_NOTIONA_minNotional", "min_qty_val_notional", "price", "min_qty", "min_qty_rounded", "min_qty_val_in_eur"]]

,min_safe_qty,min_safe_qty_val_in_eur,f_LOT_minQty,f_LOT_stepSize,f_NOTIONA_minNotional,min_qty_val_notional,price,min_qty,min_qty_rounded,min_qty_val_in_eur
symbol,,,,,,,,,,
BNBBTC,0.031000,16.938411,0.010000,0.010000,0.0001,0.000119,0.011859,0.008432,0.010000,5.464004
BNBETH,0.031000,16.938411,0.001000,0.001000,0.0050,0.005138,0.165757,0.030165,0.031000,16.938411
BNBEUR,0.031000,16.938411,0.000100,0.000100,10.0000,10.053767,546.400361,0.018302,0.018400,10.053767
BNBUSDT,0.031000,16.938411,0.000100,0.000100,10.0000,10.028421,664.133829,0.015057,0.015100,8.250645
BTCEUR,0.000217,10.007337,0.000001,0.000001,10.0000,10.007337,46116.759247,0.000217,0.000217,10.007337
BTCUSDT,0.000217,10.007337,0.000001,0.000001,10.0000,10.023506,55997.239859,0.000179,0.000179,8.254900
DOGEBTC,24.100000,10.019785,1.000000,1.000000,0.0001,0.000108,0.000009,11.086475,12.000000,4.989105
DOGEEUR,24.100000,10.019785,0.100000,0.100000,10.0000,10.019785,0.415759,24.052411,24.100000,10.019785
DOGEUSDT,24.100000,10.019785,0.100000,0.100000,10.0000,10.039943,0.504520,19.820829,19.900000,8.273599
